# Riddler Classic 

**2021-01-22**: https://fivethirtyeight.com/features/can-you-skillfully-ski-the-slopes/

## Can You Skillfully Ski The Slopes?

Congratulations, you’ve made it to the finals of the Riddler Ski Federation’s winter championship! There’s just one opponent left to beat, and then the gold medal will be yours.

Each of you will complete two runs down the mountain, and the times of your runs will be added together. Whoever skis in the least overall time is the winner. Also, this being the Riddler Ski Federation, you have been presented detailed data on both you and your opponent. You are evenly matched, and both have the same [normal](https://mathworld.wolfram.com/NormalDistribution.html) probability distribution of finishing times for each run. And for both of you, your time on the first run is completely independent of your time on the second run.

For the first runs, your opponent goes first. Then, it’s your turn. As you cross the finish line, your coach excitedly signals to you that you were faster than your opponent. Without knowing either exact time, what’s the probability that you will still be ahead after the second run and earn your gold medal?

In [1]:
import numpy as np

In [2]:
times = np.random.normal(100, 10, size=(2,2))
times

array([[102.29262231,  83.92271935],
       [ 99.19403202, 117.93461956]])

Let the first column represent the times of the skiers on their first run down the mountain, and the second column represent the second run down the slopes. We know that on the first run I was faster than my opponent, so my row will be the row containing the largest value in the first column.

I win after the second run if the row with the best (lowest) time in the first run
```python 
np.argmin(times[:, 0])
```

is the same row number with the lowest sum of the two runs.

```python
np.argmin(np.sum(times, axis=1))
```


I'll start writing a simulation with a loop and then later return to convert it into pure vectorized operations with numpy. 

In [3]:
def i_won_gold_medal(mu=100, stdev=10):
    times = np.random.normal(mu, stdev, size=(2,2))
    fastest_first_run = np.argmin(times[:, 0])
    fastest_overall = np.argmin(np.sum(times, axis=1))
    return fastest_first_run == fastest_overall

In [4]:
trials = 10**6
sum(i_won_gold_medal(mu=100, stdev=10) for _ in range(trials))/trials

0.749668

The probability I win overall, given I had the best time on the first trial, is 75%.

*Extra credit:* Over in the snowboarding championship, there are 30 finalists, including you (apparently, you’re a dual-sport threat!). Again, you are the last one to complete the first run, and your coach signals that you are in the lead. What is the probability that you’ll win gold in snowboarding?

I'll just modify the original function to take in the number of snowboarders as a parameter.

In [5]:
def i_won_gold_medal(snowboarders=2, mu=100, stdev=10):
    times = np.random.normal(mu, stdev, size=(snowboarders, 2))
    fastest_first_run = np.argmin(times[:, 0])
    fastest_overall = np.argmin(np.sum(times, axis=1))
    return fastest_first_run == fastest_overall

In [6]:
trials = 10**6
sum(i_won_gold_medal(snowboarders=30, mu=100, stdev=10) for _ in range(trials))/trials

0.314214

With 30 partipants total the probability of winning has decreased to ~31.5%. 

Alright but instead of using a loop in a simulation I'd really prefer to do this solely with numpy. I don't use 3 dimensional arrays often though so this may take some tinkering. 

I'm going to start by simulating 2 skiers with 2 times each but four competitions total, with dimensions (4, 2, 2). 

In [7]:
times = np.random.normal(100, 10, size=(4, 2, 2))
times

array([[[ 97.75440684,  86.42984797],
        [112.96338887, 109.62303099]],

       [[107.66435836, 107.65011931],
        [ 99.21201424,  89.51882512]],

       [[102.96747223, 100.01737636],
        [ 97.54001381,  92.36475469]],

       [[107.81258434, 111.36392443],
        [ 95.40208966, 107.56608638]]])

In [8]:
times[:, :, 0]

array([[ 97.75440684, 112.96338887],
       [107.66435836,  99.21201424],
       [102.96747223,  97.54001381],
       [107.81258434,  95.40208966]])

In [9]:
fastest_first_run = np.argmin(times[:, :, 0], axis=1)
fastest_first_run

array([0, 1, 1, 1], dtype=int64)

In [10]:
np.sum(times, axis=2)

array([[184.18425482, 222.58641985],
       [215.31447767, 188.73083937],
       [202.98484858, 189.9047685 ],
       [219.17650877, 202.96817604]])

In [11]:
np.argmin(np.sum(times, axis=2), axis=1)

array([0, 1, 1, 1], dtype=int64)

In [12]:
np.argmin(times[:, :, 0], axis=1) == np.argmin(np.sum(times, axis=2), axis=1)

array([ True,  True,  True,  True])

Putting it all together...

In [13]:
simulations = 10**6
competitors = 30
mu = 100
stdev = 10

times = np.random.normal(mu, stdev, size=(simulations, competitors, 2))
fastest_first_run = np.argmin(times[:, :, 0], axis=1)
fastest_overall = np.argmin(np.sum(times, axis=2), axis=1)
np.mean(fastest_first_run == fastest_overall)

0.314561

There we go, the fully vectorized code runs significantly faster than my previous function that required a loop.